Code for Parts 1 and 2: Strategy Replication and Family of Strategies

In [11]:
#Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
#Import Data
contdf = pd.read_excel(
    "contrarian_data.xlsx",
    header = 0,
    usecols = ["d", "id", "r"],
    parse_dates = ["d"]
)
contdf["r_simple"] = np.exp(contdf["r"]) - 1
print(contdf.isna().sum())
print(contdf.head(10))

d           0
id          0
r           0
r_simple    0
dtype: int64
           d     id         r  r_simple
0 1992-01-02  10137  0.000000  0.000000
1 1992-01-02  10225 -0.016800 -0.016660
2 1992-01-02  10401  0.003190  0.003195
3 1992-01-02  10488  0.117783  0.125000
4 1992-01-02  10516 -0.038500 -0.037768
5 1992-01-02  10656  0.031100  0.031589
6 1992-01-02  10866  0.057400  0.059079
7 1992-01-02  10890  0.114410  0.121212
8 1992-01-02  11308  0.000000  0.000000
9 1992-01-02  11404 -0.008770 -0.008732


In [ ]:
#Pivot the data wide so matrix arithmetic in contrarian strategy is easier
widecontdf = contdf.pivot(index = "d", columns = "id", values = "r_simple")
widecontdf["r_bar"] = widecontdf.mean(axis = 1)

#Full Excess Returns DataFrame
r_excessdf = widecontdf.sub(widecontdf["r_bar"], axis = 0)
r_excessdf["r_bar"] = widecontdf["r_bar"]
r_excessdf["n"] = r_excessdf.count(axis = 1) - 1 #subtract 1 because 1 column is r_bar
#TODO: n changes from 690 to 689 somewhere, want to find and observe anomaly
print(r_excessdf.head(10))

id             10137     10225     10401     10488     10516     10656  \
d                                                                        
1992-01-02  0.001317 -0.015343  0.004512  0.126317 -0.036452  0.032905   
1992-01-03 -0.005176  0.000486  0.001565 -0.063585 -0.011912 -0.069234   
1992-01-06  0.003949 -0.018457  0.008122 -0.063257 -0.016286  0.039064   
1992-01-07 -0.004664 -0.010411  0.019947 -0.001888 -0.025797 -0.001888   
1992-01-08 -0.011472  0.017025 -0.008943  0.150352 -0.018123 -0.005898   
1992-01-09 -0.007582 -0.010380  0.001343 -0.004776  0.028282 -0.004776   
1992-01-10 -0.004034 -0.004054 -0.001455  0.001580  0.009582  0.001580   
1992-01-13 -0.025938 -0.009008 -0.000514 -0.000514  0.011356 -0.000514   
1992-01-14 -0.013164 -0.001702 -0.013313 -0.145403 -0.014190 -0.010268   
1992-01-15 -0.029558  0.002088 -0.012112  0.144138  0.019373 -0.032991   

id             10866     10890     11308     11404  ...     81700     81702  \
d                               

In [20]:
#Truncate data to make it easier to work with during development and debugging before applying to whole dataset
stock_cols_small = np.random.choice(range(1, 690), size = 10, replace = False)
stock_cols_small.sort()
print(stock_cols_small)

small_r_df = r_excessdf.iloc[:, stock_cols_small]
print(small_r_df)

[ 26  63 280 282 311 398 592 607 608 650]
id             12570     15580     33612     34121     37807     46674  \
d                                                                        
1992-01-02 -0.016031  0.251317 -0.044691  0.027658  0.016328 -0.006374   
1992-01-03 -0.025633 -0.074663 -0.001962  0.000556  0.040761 -0.007990   
1992-01-06 -0.004456 -0.004456 -0.010448 -0.004456 -0.010518 -0.012206   
1992-01-07 -0.001888  0.069549 -0.038019  0.099752 -0.003916 -0.012333   
1992-01-08  0.027756 -0.005898 -0.005898  0.024866 -0.003866 -0.008535   
...              ...       ...       ...       ...       ...       ...   
1997-12-24 -0.011387 -0.022676 -0.011881  0.001134  0.001134  0.004941   
1997-12-26 -0.004567  0.024067 -0.009097 -0.009177 -0.010474  0.003471   
1997-12-29  0.035709 -0.003024 -0.008972 -0.008972  0.006750  0.000482   
1997-12-30 -0.022154 -0.008090 -0.009578 -0.011153  0.012231 -0.004644   
1997-12-31  0.023258 -0.054563  0.014521  0.005883 -0.005544 -0.000088